In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import sys

print(f'PID number is {os.getpid()}')

sys.path.append(r'C:\Users\cosmosyw\Documents\Softwares')

pos_cols = ['z_um', 'x_um', 'y_um']
import ImageAnalysis3 as ia
from ImageAnalysis3 import *
from ImageAnalysis3.classes import _allowed_kwds

import h5py

PID number is 20444


# Load chr zxy list

In [2]:
# trans
# the calculated AB desnity ratio for single-cell
exp_folder = r'E:\DNA_analysis\Postanalysis_MeCP2'
chr_zxy_folder = exp_folder
chr_zxy_fname = os.path.join(chr_zxy_folder , r'subclass_2_chr2Zxys.pkl')

class_2_chrZxysList = pickle.load(open(chr_zxy_fname, 'rb'))

print(class_2_chrZxysList.keys())

dict_keys(['Oligo_WT', 'Astro_WT', 'Peri_WT', 'L6b_WT', 'L6b_KO', 'L6 CT_KO', 'Astro_KO', 'L6 CT_WT', 'Endo-PVM_WT', 'OPC_KO', 'Pvalb_KO', 'Pvalb_WT', 'Peri_KO', 'L6 IT_KO', 'Vip_WT', 'Sncg_WT', 'Lamp5_KO', 'Sst_KO', 'L6 IT_WT', 'Sncg_KO', 'Oligo_KO', 'OPC_WT', 'Sst_WT', 'Vip_KO', 'SMC-VLMC_KO', 'SMC-VLMC_WT', 'L5/6 NP_WT', 'L5/6 NP_KO', 'L5 ET_KO', 'L5 ET_WT', 'Endo-PVM_KO', 'Micro_KO', 'L5 IT_WT', 'Micro_WT', 'L5 IT_KO', 'Lamp5_WT', 'L4/5 IT_WT', 'L4/5 IT_KO', 'L2/3 IT_WT', 'L2/3 IT_KO'])


# Calculate radial positioning

In [3]:

# quick get cell pts as zxy arr
def chrZXY_to_cell_pts_arr (chrZxysList_cell:dict):
    cell_pts = []
    for _chrom, _chr_zxys in chrZxysList_cell.items():
        for _fiberidx, _ichr_zxys in enumerate(_chr_zxys):
            cell_pts.extend(_ichr_zxys)
    return cell_pts   

def get_intersection(U,hull):
    eq=hull.equations.T
    V,b=eq[:-1],eq[-1]
    alpha=-b/np.dot(U,V)
    return U*np.full(np.array(U).shape, np.min(alpha[alpha>0]))

def Radial_positioning_from_center_for_loci (chr_2_zxys_list,
                                 function='nanmedian', axis=0, 
                                data_num_th=100,  # nonnan data num th for nanmedian along axis
                                Zxys_num_min_th=600,  # total least number of nonnan data Zxys for using convex hull 
                               ):
    
    from scipy.spatial.distance import euclidean, cdist, pdist
    from scipy.spatial import ConvexHull
    import math
    
    # 1. prepare dict to store loci-2-center dist results
    _out_loci_2_center_dist_dict = {}
    _out_loci_radial_position_dict = {}
    # loop through cells
    for _cell_index, _cell_chr_2_zxys_dict in enumerate(chr_2_zxys_list):
        # 1.1. get nuclei centroid and radius
        combined_cell_chrZxy_list = chrZXY_to_cell_pts_arr(_cell_chr_2_zxys_dict)
        # use all non-nan points to calculate convex hull            
        nonnan_index =np.where(np.sum(~np.isnan(combined_cell_chrZxy_list)>0,axis=1))
        if (len(nonnan_index[0])>=Zxys_num_min_th):
            points=np.array(combined_cell_chrZxy_list)[nonnan_index]
            hull = ConvexHull(points)  
            # get hull centroid
            cz = np.mean(hull.points[hull.vertices,0])
            cx = np.mean(hull.points[hull.vertices,1])
            cy = np.mean(hull.points[hull.vertices,2])
            # attention: zxy in correct order!!
            _hull_ct_zxy = np.array([cz,cx,cy])
            # update point and hull
            points = points - _hull_ct_zxy
            hull = ConvexHull(points)

            # 1.2. caluclate normalized dist to nuclei centroid for radial positioning if applicable
            # loop through chr
            for _chr_key, _chr_zxys in _cell_chr_2_zxys_dict.items():
                # ini chr_key if not added already
                if _chr_key not in _out_loci_2_center_dist_dict.keys():
                    _out_loci_2_center_dist_dict[_chr_key]=[]
                    _out_loci_radial_position_dict[_chr_key]=[]
                # loop through haploid
                for _ichr_zxys in _chr_zxys:
                    #center the zxy list
                    ichr_zxys = _ichr_zxys - _hull_ct_zxy
                    
                    # define output
                    _loci_2_center_list = []
                    _loci_rp = []
                    
                    #loop through every point
                    for _zxys in ichr_zxys:
                        if len(np.where(np.isnan(_zxys))[0])>0:
                            _loci_2_center_list.append(np.nan)
                            _loci_rp.append(np.nan)
                        else:
                            dist_to_center = euclidean(_zxys, [0,0,0])
                            _loci_2_center_list.append(dist_to_center)
                            edge_pt = get_intersection(_zxys, hull)
                            _loci_rp.append(dist_to_center/euclidean(edge_pt, [0,0,0]))
                    # append dist list array for each haploid 
                    #print('cell added')
                    _out_loci_2_center_dist_dict[_chr_key].append(_loci_2_center_list)
                    _out_loci_radial_position_dict[_chr_key].append(_loci_rp)
                
    # get nanmedian for each dist or return all dist            

    _summary_dict_dist={}
    for _chr_key, _dists_list in _out_loci_2_center_dist_dict.items():
        if len(_dists_list) >= 0: # discard if not enough cell statistics
            _dists_array = np.array(_dists_list)
            _summary_result = np.nanmedian(_dists_array, axis=axis)
            _summary_dict_dist[_chr_key] = _summary_result
    _summary_dict_rp={}
    for _chr_key, _dists_list in _out_loci_radial_position_dict.items():
        if len(_dists_list) >= 0: # discard if not enough cell statistics
            _dists_array = np.array(_dists_list)
            _summary_result = np.nanmedian(_dists_array, axis=axis)
            _summary_dict_rp[_chr_key] = _summary_result

    return _summary_dict_dist, _summary_dict_rp

In [4]:
result_dist_to_center ={}
result_rp = {}

for _class, _class_list in class_2_chrZxysList.items():
    print (f'-- Computing for {_class}.')
    result_dist_to_center[_class], result_rp[_class]=Radial_positioning_from_center_for_loci (_class_list,
                                 function='nanmedian', axis=0, 
                                data_num_th=100)  # nonnan data num th for nanmedian along axis)

-- Computing for Oligo_WT.
-- Computing for Astro_WT.
-- Computing for Peri_WT.
-- Computing for L6b_WT.
-- Computing for L6b_KO.
-- Computing for L6 CT_KO.
-- Computing for Astro_KO.
-- Computing for L6 CT_WT.
-- Computing for Endo-PVM_WT.
-- Computing for OPC_KO.
-- Computing for Pvalb_KO.
-- Computing for Pvalb_WT.
-- Computing for Peri_KO.
-- Computing for L6 IT_KO.
-- Computing for Vip_WT.
-- Computing for Sncg_WT.
-- Computing for Lamp5_KO.
-- Computing for Sst_KO.
-- Computing for L6 IT_WT.
-- Computing for Sncg_KO.
-- Computing for Oligo_KO.
-- Computing for OPC_WT.
-- Computing for Sst_WT.
-- Computing for Vip_KO.
-- Computing for SMC-VLMC_KO.
-- Computing for SMC-VLMC_WT.
-- Computing for L5/6 NP_WT.
-- Computing for L5/6 NP_KO.
-- Computing for L5 ET_KO.
-- Computing for L5 ET_WT.
-- Computing for Endo-PVM_KO.
-- Computing for Micro_KO.
-- Computing for L5 IT_WT.
-- Computing for Micro_WT.
-- Computing for L5 IT_KO.
-- Computing for Lamp5_WT.
-- Computing for L4/5 IT_WT.
-- 

# Add radial positioning with AB

In [5]:
# load the file 
df_ab = pd.read_csv(r"E:\DNA_analysis\Postanalysis_MeCP2\AB_assignment_shared_ABth_zero.csv")
df_ab

,loci_name,Astro,Endo,GABA,Gluta,Micro,Oligo
0,chr1_3742742_3759944,B,B,B,B,B,B
1,chr1_6245958_6258969,A,B,B,B,B,B
2,chr1_8740008_8759916,B,B,B,B,B,B
3,chr1_9627926_9637875,A,B,B,B,A,B
4,chr1_9799472_9811359,A,B,B,B,A,A
...,...,...,...,...,...,...,...
1976,chrX_163750534_163758706,A,A,A,A,A,A
1977,chrX_166247682_166259932,A,A,A,A,A,A
1978,chrX_167157164_167167452,A,A,A,A,A,A
1979,chrX_168746045_168757590,A,A,A,A,A,A


## Generate AB dataframe for each class

In [ ]:
class_2_subclass_names = {
    'Gluta':['L6b', 'L6 CT', 'L6 IT', 'L5 IT', 'L4/5 IT', 'L2/3 IT', 'L5/6 NP', 'L5 ET', ],
    'GABA':['Sncg', 'Sst', 'Vip', 'Pvalb', 'Lamp5', ],
    'Astro':['Astro', ],
    'Endo':['Endo-PVM', 'SMC-VLMC', 'Peri'],
    'Micro':['Micro', ],
    'Oligo':['Oligo', 'OPC', ],
}

subclasses = []
subclass_to_majortype = {}
for majortype, subcls_list in class_2_subclass_names.items():
    for subcls in subcls_list:
        subclasses.append(subcls)
        subclass_to_majortype[subcls] = majortype

In [ ]:
majorTypes = ['Gluta', 'GABA', 'Astro', 'Oligo', 'Micro', 'Endo']
ordered_chr_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9','10', '11', '12', '13', '14', '15', '16', '17', '18', '19','X']

output_folder = r'E:\DNA_analysis\Postanalysis_MeCP2\radial_position'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [ ]:
df_list = []

for cls in subclasses:
        
    df_cls = df_ab.loc[:,['loci_name', subclass_to_majortype[cls]]].copy()
    
    wt_positions = []
    ko_positions = []
    
    wt_center_dist = []
    ko_center_dist = []
    
    for _chr_key in ordered_chr_list:
        wt_dist_list =  result_rp[cls+'_WT'][_chr_key]
        ko_dist_list =  result_rp[cls+'_KO'][_chr_key]
        wt_center_dist_list =  result_dist_to_center[cls+'_WT'][_chr_key]
        ko_center_dist_list =  result_dist_to_center[cls+'_KO'][_chr_key]

        for wt_dist, ko_dist in zip(wt_dist_list, ko_dist_list):
            wt_positions.append(wt_dist)
            ko_positions.append(ko_dist)
        for wt_dist, ko_dist in zip(wt_center_dist_list, ko_center_dist_list):
            wt_center_dist.append(wt_dist)
            ko_center_dist.append(ko_dist)
    
    df_cls['WT_norm_RP'] = wt_positions
    df_cls['KO_norm_RP'] = ko_positions
    df_cls['WT_distance_to_center'] = wt_center_dist
    df_cls['KO_distance_to_center'] = ko_center_dist
    
    df_cls.rename(columns={subclass_to_majortype[cls]:'AB'}, inplace=True)
    
    df_cls['subclass'] = cls
    df_list.append(df_cls)

df_subclass_rp = pd.concat(df_list, ignore_index=True)
df_subclass_rp.head()

In [ ]:
df_subclass_rp.to_csv(os.path.join(output_folder, 'all_subclass_rp_600pts_final.csv'))